# Protocol: Improving State Space Coverage using Start Overs

Date: 17.09.2021

## Question

Do *Start Overs* increase the State Space Coverage Growth and thus the model checking performance?

## Hypothesis

The basic BFS search terminates as soon as all successors are already visited according to the hash table. Thus, the maximum BFS depth is determined by the hash table's capacity. Also, each VT again starts at the initial state, resulting in all VTs potentially only exploring the tip of a model.

By continuing the BFS within a VT using the set of last unvisited successors, we can reach deeper states of the model. By exploring deeper states, we expect to discover more violations and thus a faster growing state space coverage.

## Setup

- GPU: NVIDIA GeForce RTX 2080 Ti
- Program: `main` branch, commit e160572
- Model: Waypoints model
- CUDA_FLAGS: `-DGRAPPLE_MODEL=WaypointsState`

## Implementation

First, we compile with the additional CUDA_FLAGS `-DGRAPPLE_HT=18 -DGRAPPLE_SO=0` (the default parameters).

```
$ time ./build/grapple -s 1736331306 -n 1000
...

real    7m22.195s
user    7m21.844s
sys     0m0.296s
```

Full output data is available at [EXP-04-start-overs-1.csv](./data/EXP-04-start-overs-1.csv).

---

For the second experiment, we compile with the additional CUDA_FLAGS `-DGRAPPLE_HT=14 -DGRAPPLE_SO=15`.

```
$ time ./build/grapple -s 1736331306 -n 1000

^C

real    28m31.640s
user    28m30.974s
sys     0m0.600s
```

Stopped after 244 runs (61000 VTs).

Full output data is available at [EXP-04-start-overs-2.csv](./data/EXP-04-start-overs-2.csv).

## Evaluation

## Conclusion, Discussion